In [1]:
import tensorflow as tf
import numpy as np
import os
slim = tf.contrib.slim


In [2]:
SEQ_LEN = 10 # this parameter can be changed. TODO try longer sequences if memory is available.
BATCH_SIZE = 4 # this parameter can also be changed
LEFT_CONTEXT = 5

HEIGHT = 480
WIDTH = 640
CHANNELS = 3

RNN_SIZE = 32
RNN_PROJ = 32

CSV_HEADER = "index,timestamp,width,height,frame_id,filename,angle,torque,speed,lat,long,alt".split(",")
OUTPUTS = CSV_HEADER[-6:-3] # angle,torque,speed
OUTPUT_DIM = len(OUTPUTS) # predict all features

In [3]:
class BatchGenerator(object):
    def __init__(self, sequence, seq_len, batch_size):
        self.sequence = sequence
        self.seq_len = seq_len
        self.batch_size = batch_size
        chunk_size = 1 + (len(sequence) - 1) / batch_size
        self.indices = [(i*chunk_size) % len(sequence) for i in range(batch_size)]
        
    def next(self):
        while True:
            output = []
            for i in range(self.batch_size):
                idx = self.indices[i]
                left_pad = self.sequence[idx - LEFT_CONTEXT:idx]
                if len(left_pad) < LEFT_CONTEXT:
                    left_pad = [self.sequence[0]] * (LEFT_CONTEXT - len(left_pad)) + left_pad
                assert len(left_pad) == LEFT_CONTEXT
                leftover = len(self.sequence) - idx
                if leftover >= self.seq_len:
                    result = self.sequence[idx:idx + self.seq_len]
                else:
                    result = self.sequence[idx:] + self.sequence[:self.seq_len - leftover]
                assert len(result) == self.seq_len
                self.indices[i] = (idx + self.seq_len) % len(self.sequence)
                images, targets = zip(*result)
                images_left_pad, _ = zip(*left_pad)
                output.append((np.stack(images_left_pad + images), np.stack(targets)))
            output = zip(*output)
            output[0] = np.stack(output[0]) # batch_size x (LEFT_CONTEXT + seq_len)
            output[1] = np.stack(output[1]) # batch_size x seq_len x OUTPUT_DIM
            return output
        
def read_csv(filename):
    with open(filename, 'r') as f:
        lines = [ln.strip().split(",")[-7:-3] for ln in f.readlines()]
        lines = map(lambda x: (x[0], np.float32(x[1:])), lines) # imagefile, outputs
        return lines

def process_csv(filename, val=5):
    sum_f = np.float128([0.0] * OUTPUT_DIM)
    sum_sq_f = np.float128([0.0] * OUTPUT_DIM)
    lines = read_csv(filename)
    # leave val% for validation
    train_seq = []
    valid_seq = []
    cnt = 0
    for ln in lines:
        if cnt < SEQ_LEN * BATCH_SIZE * (100 - val): 
            train_seq.append(ln)
            sum_f += ln[1]
            sum_sq_f += ln[1] * ln[1]
        else:
            valid_seq.append(ln)
        cnt += 1
        cnt %= SEQ_LEN * BATCH_SIZE * 100
    mean = sum_f / len(train_seq)
    var = sum_sq_f / len(train_seq) - mean * mean
    std = np.sqrt(var)
    print len(train_seq), len(valid_seq)
    print mean, std
    return (train_seq, valid_seq), (mean, std)

In [4]:
(train_seq_main_1, valid_seq_1), (mean_1, std_1) = process_csv(filename="output/dataset5.csv", val=5)
(train_seq_main_2, valid_seq_2), (mean_2, std_2) = process_csv(filename="output/dataset6.csv", val=0)
(train_seq_main_3, valid_seq_3), (mean_3, std_3) = process_csv(filename="output/dataset7.csv", val=0)
(train_seq_main_4, valid_seq_4), (mean_4, std_4) = process_csv(filename="output/dataset8.csv", val=0)

mean, std = mean_1, std_1

add_sets = [] # ["output/dataset1.csv", "output/dataset2.csv", "output/dataset3.csv", "output/dataset4.csv"]
train_seq_add = map(read_csv, add_sets)

train_seq = sum(train_seq_add, []) + train_seq_main_1 + train_seq_main_2 + train_seq_main_3 + train_seq_main_4
valid_seq = (valid_seq_1 + valid_seq_2 + valid_seq_3 + valid_seq_4)

test_seq = read_csv("challenge_2/exampleSubmissionInterpolatedFinal.csv")

print len(test_seq)

139758 7200
[-0.0047997167  0.043900186  11.352226] [ 0.051920877  0.39605346  3.7726]
154131 0
[ 0.0017989723  0.092735196  10.134581] [ 0.048207131  0.3890369  4.0389421]
33808 0
[-0.0084786136 -0.09063468  15.625128] [ 0.27152468  0.78744243  5.6971225]
89205 0
[-7.899939e-05  0.085713923  10.784245] [ 0.085205067  0.38865169  4.6383147]
5614


In [5]:
layer_norm = lambda x: tf.contrib.layers.layer_norm(inputs=x, center=True, scale=True, activation_fn=None, trainable=True)

def get_optimizer(loss, lrate):
        optimizer = tf.train.AdamOptimizer(learning_rate=lrate)
        gradvars = optimizer.compute_gradients(loss)
        gradients, v = zip(*gradvars)
        print [x.name for x in v]
        gradients, _ = tf.clip_by_global_norm(gradients, 15.0)
        return optimizer.apply_gradients(zip(gradients, v))

def apply_vision_simple(image, keep_prob, batch_size, seq_len, scope=None, reuse=None):
    video = tf.reshape(image, shape=[batch_size, LEFT_CONTEXT + seq_len, HEIGHT, WIDTH, CHANNELS])
    with tf.variable_scope(scope, 'Vision', [image], reuse=reuse):
            net = slim.convolution(video, num_outputs=64, kernel_size=[3,12,12], stride=[1,6,6], padding="VALID")
            net = tf.nn.dropout(x=net, keep_prob=keep_prob)
            aux1 = slim.fully_connected(tf.reshape(net[:, -seq_len:, :, :, :], [batch_size, seq_len, -1]), 128, activation_fn=None)
            net = slim.convolution(net, num_outputs=64, kernel_size=[2,5,5], stride=[1,2,2], padding="VALID")
            net = tf.nn.dropout(x=net, keep_prob=keep_prob)
            aux2 = slim.fully_connected(tf.reshape(net[:, -seq_len:, :, :, :], [batch_size, seq_len, -1]), 128, activation_fn=None)
            net = slim.convolution(net, num_outputs=64, kernel_size=[2,5,5], stride=[1,1,1], padding="VALID")
            net = tf.nn.dropout(x=net, keep_prob=keep_prob)
            aux3 = slim.fully_connected(tf.reshape(net[:, -seq_len:, :, :, :], [batch_size, seq_len, -1]), 128, activation_fn=None)
            net = slim.convolution(net, num_outputs=64, kernel_size=[2,5,5], stride=[1,1,1], padding="VALID")
            net = tf.nn.dropout(x=net, keep_prob=keep_prob)
            print net # TODO must be batch_size x seq_len x ...
            aux4 = slim.fully_connected(tf.reshape(net, [batch_size, seq_len, -1]), 128, activation_fn=None)
            net = slim.fully_connected(tf.reshape(net, [batch_size, seq_len, -1]), 1024, activation_fn=tf.nn.relu)
            net = tf.nn.dropout(x=net, keep_prob=keep_prob)
            net = slim.fully_connected(net, 512, activation_fn=tf.nn.relu)
            net = tf.nn.dropout(x=net, keep_prob=keep_prob)
            net = slim.fully_connected(net, 256, activation_fn=tf.nn.relu)
            net = tf.nn.dropout(x=net, keep_prob=keep_prob)
            net = slim.fully_connected(net, 128, activation_fn=None)
            return layer_norm(tf.nn.elu(net + aux1 + aux2 + aux3 + aux4))

class SamplingRNNCell(tf.nn.rnn_cell.RNNCell):
  """Simple sampling RNN cell."""

  def __init__(self, num_outputs, use_ground_truth, internal_cell, keep_prob):
    """
    if use_ground_truth then don't sample
    """
    self._num_outputs = num_outputs
    self._use_ground_truth = use_ground_truth
    self._internal_cell = internal_cell
    self._keep_prob = keep_prob
  
  @property
  def state_size(self):
    return self._num_outputs, self._internal_cell.state_size # previous output and bottleneck state

  @property
  def output_size(self):
    return self._num_outputs

  def __call__(self, inputs, state, scope=None):
    (visual_feats, current_ground_truth) = inputs
    prev_output, prev_state_internal = state
    # the following is just for a baseline
    context = tf.concat(1, [prev_output, visual_feats])
    new_output_internal, new_state_internal = internal_cell(context, prev_state_internal)
    new_output = tf.contrib.layers.fully_connected(
        inputs=tf.concat(1, [new_output_internal, prev_output, visual_feats]),
        num_outputs=self._num_outputs,
        activation_fn=None,
        scope="OutputProjection")
    return new_output, (current_ground_truth if self._use_ground_truth else new_output, new_state_internal)
    
    

In [6]:
graph = tf.Graph()

with graph.as_default():
    #inputs  
    learning_rate = tf.placeholder_with_default(input=1e-4, shape=())
    keep_prob = tf.placeholder_with_default(input=1.0, shape=())
    
    inputs = tf.placeholder(shape=(BATCH_SIZE,LEFT_CONTEXT+SEQ_LEN), dtype=tf.string) # pathes to png files from the central camera
    targets = tf.placeholder(shape=(BATCH_SIZE,SEQ_LEN,OUTPUT_DIM), dtype=tf.float32) # seq_len x batch_size x OUTPUT_DIM
    targets_normalized = (targets - mean) / std
    
    input_images = tf.pack([tf.image.decode_png(tf.read_file(x))
                            for x in tf.unpack(tf.reshape(inputs, shape=[(LEFT_CONTEXT+SEQ_LEN) * BATCH_SIZE]))])
    input_images = -1.0 + 2.0 * tf.cast(input_images, tf.float32) / 255.0
    input_images.set_shape([(LEFT_CONTEXT+SEQ_LEN) * BATCH_SIZE, HEIGHT, WIDTH, CHANNELS])
    visual_conditions_reshaped = apply_vision_simple(image=input_images, keep_prob=keep_prob, 
                                                     batch_size=BATCH_SIZE, seq_len=SEQ_LEN)
    visual_conditions = tf.reshape(visual_conditions_reshaped, [BATCH_SIZE, SEQ_LEN, -1])
    visual_conditions = tf.nn.dropout(x=visual_conditions, keep_prob=keep_prob)
    
    rnn_inputs_with_ground_truth = (visual_conditions, targets_normalized)
    rnn_inputs_autoregressive = (visual_conditions, tf.zeros(shape=(BATCH_SIZE, SEQ_LEN, OUTPUT_DIM), dtype=tf.float32))
    
    internal_cell = tf.nn.rnn_cell.LSTMCell(num_units=RNN_SIZE, num_proj=RNN_PROJ)
    cell_with_ground_truth = SamplingRNNCell(num_outputs=OUTPUT_DIM, use_ground_truth=True, internal_cell=internal_cell, keep_prob=keep_prob)
    cell_autoregressive = SamplingRNNCell(num_outputs=OUTPUT_DIM, use_ground_truth=False, internal_cell=internal_cell, keep_prob=keep_prob)
    
    def get_initial_state(complex_state_tuple_sizes):
        flat_sizes = tf.nn.rnn_cell.nest.flatten(complex_state_tuple_sizes)
        init_state_flat = [tf.tile(
            multiples=[BATCH_SIZE, 1], 
            input=tf.get_variable("controller_initial_state_%d" % i, initializer=tf.zeros_initializer, shape=([1, s]), dtype=tf.float32))
         for i,s in enumerate(flat_sizes)]
        init_state = tf.nn.rnn_cell.nest.pack_sequence_as(complex_state_tuple_sizes, init_state_flat)
        return init_state
    def deep_copy_initial_state(complex_state_tuple):
        flat_state = tf.nn.rnn_cell.nest.flatten(complex_state_tuple)
        flat_copy = [tf.identity(s) for s in flat_state]
        deep_copy = tf.nn.rnn_cell.nest.pack_sequence_as(complex_state_tuple, flat_copy)
        return deep_copy
    
    controller_initial_state_variables = get_initial_state(cell_autoregressive.state_size)
    controller_initial_state_autoregressive = deep_copy_initial_state(controller_initial_state_variables)
    controller_initial_state_gt = deep_copy_initial_state(controller_initial_state_variables)

    with tf.variable_scope("predictor"):
        out_gt, controller_final_state_gt = tf.nn.dynamic_rnn(cell=cell_with_ground_truth, inputs=rnn_inputs_with_ground_truth, 
                          sequence_length=[SEQ_LEN]*BATCH_SIZE, initial_state=controller_initial_state_gt, dtype=tf.float32,
                          swap_memory=True, time_major=False)
    with tf.variable_scope("predictor", reuse=True):
        out_autoregressive, controller_final_state_autoregressive = tf.nn.dynamic_rnn(cell=cell_autoregressive, inputs=rnn_inputs_autoregressive, 
                          sequence_length=[SEQ_LEN]*BATCH_SIZE, initial_state=controller_initial_state_autoregressive, dtype=tf.float32,
                          swap_memory=True, time_major=False)
    
    mse_gt = tf.reduce_mean(tf.squared_difference(out_gt, targets_normalized))
    mse_autoregressive = tf.reduce_mean(tf.squared_difference(out_autoregressive, targets_normalized))
    mse_autoregressive_steering = tf.reduce_mean(tf.squared_difference(out_autoregressive[:, :, 0], targets_normalized[:, :, 0]))
    steering_predictions = (out_autoregressive[:, :, 0] * std[0]) + mean[0]
    
    total_loss = mse_autoregressive_steering # + 0.1 * (mse_gt + mse_autoregressive)
    
    optimizer = get_optimizer(total_loss, learning_rate)

    tf.scalar_summary("MAIN TRAIN METRIC: rmse_autoregressive_steering", tf.sqrt(mse_autoregressive_steering))
    tf.scalar_summary("rmse_gt", tf.sqrt(mse_gt))
    tf.scalar_summary("rmse_autoregressive", tf.sqrt(mse_autoregressive))
    
    summaries = tf.merge_all_summaries()
    train_writer = tf.train.SummaryWriter('v3/train_summary', graph=graph)
    valid_writer = tf.train.SummaryWriter('v3/valid_summary', graph=graph)
    saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
   

Tensor("Vision/dropout_3/mul:0", shape=(4, 10, 30, 43, 64), dtype=float32)
[u'Vision/Conv/weights:0', u'Vision/Conv/biases:0', u'Vision/fully_connected/weights:0', u'Vision/fully_connected/biases:0', u'Vision/Conv_1/weights:0', u'Vision/Conv_1/biases:0', u'Vision/fully_connected_1/weights:0', u'Vision/fully_connected_1/biases:0', u'Vision/Conv_2/weights:0', u'Vision/Conv_2/biases:0', u'Vision/fully_connected_2/weights:0', u'Vision/fully_connected_2/biases:0', u'Vision/Conv_3/weights:0', u'Vision/Conv_3/biases:0', u'Vision/fully_connected_3/weights:0', u'Vision/fully_connected_3/biases:0', u'Vision/fully_connected_4/weights:0', u'Vision/fully_connected_4/biases:0', u'Vision/fully_connected_5/weights:0', u'Vision/fully_connected_5/biases:0', u'Vision/fully_connected_6/weights:0', u'Vision/fully_connected_6/biases:0', u'Vision/fully_connected_7/weights:0', u'Vision/fully_connected_7/biases:0', u'Vision/LayerNorm/beta:0', u'Vision/LayerNorm/gamma:0', u'controller_initial_state_0:0', u'cont

In [7]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)

checkpoint_dir = os.getcwd() + "/v3"

global_train_step = 0
global_valid_step = 0
global_valid_predictions = {}

KEEP_PROB_TRAIN = 0.25

def do_epoch(session, sequences, mode):
    global global_train_step, global_valid_step, global_valid_predictions
    test_predictions = {}
    batch_generator = BatchGenerator(sequence=sequences, seq_len=SEQ_LEN, batch_size=BATCH_SIZE)
    total_num_steps = 1 + (batch_generator.indices[1] - 1) / SEQ_LEN
    controller_final_state_gt_cur, controller_final_state_autoregressive_cur = None, None
    acc_loss = np.float128(0.0)
    for step in range(total_num_steps):
        feed_inputs, feed_targets = batch_generator.next()
        feed_dict = {inputs : feed_inputs, targets : feed_targets}
        if controller_final_state_autoregressive_cur is not None:
            feed_dict.update({controller_initial_state_autoregressive : controller_final_state_autoregressive_cur})
        if controller_final_state_gt_cur is not None:
            feed_dict.update({controller_final_state_gt : controller_final_state_gt_cur})
        if mode == "train":
            feed_dict.update({keep_prob : KEEP_PROB_TRAIN})
            summary, _, loss, controller_final_state_gt_cur, controller_final_state_autoregressive_cur = \
                session.run([summaries, optimizer, mse_autoregressive_steering, controller_final_state_gt, controller_final_state_autoregressive],
                           feed_dict = feed_dict)
            train_writer.add_summary(summary, global_train_step)
            global_train_step += 1
        elif mode == "valid":
            model_predictions, summary, loss, controller_final_state_autoregressive_cur = \
                session.run([steering_predictions, summaries, mse_autoregressive_steering, controller_final_state_autoregressive],
                           feed_dict = feed_dict)
            valid_writer.add_summary(summary, global_valid_step)
            global_valid_step += 1
            
            feed_inputs = feed_inputs[:, LEFT_CONTEXT:].flatten()
            steering_targets = feed_targets[:, :, 0].flatten()
            model_predictions = model_predictions.flatten()
            stats = np.stack([steering_targets, model_predictions, (steering_targets - model_predictions)**2])
            for i, img in enumerate(feed_inputs):
                global_valid_predictions[img] = stats[:, i]
        elif mode == "test":
            model_predictions, controller_final_state_autoregressive_cur = \
                session.run([steering_predictions, controller_final_state_autoregressive],
                           feed_dict = feed_dict)           
            feed_inputs = feed_inputs[:, LEFT_CONTEXT:].flatten()
            model_predictions = model_predictions.flatten()
            for i, img in enumerate(feed_inputs):
                test_predictions[img] = model_predictions[i]
        if mode != "test":
            acc_loss += loss
            print '\r', step + 1, "/", total_num_steps, np.sqrt(acc_loss / (step+1)),
    print
    return np.sqrt(acc_loss / total_num_steps) if mode != "test" else test_predictions
    

NUM_EPOCHS=100

best_validation_score = None
with tf.Session(graph=graph, config=tf.ConfigProto(gpu_options=gpu_options)) as session:
    session.run(tf.initialize_all_variables())
    print 'Initialized'
    ckpt = tf.train.latest_checkpoint(checkpoint_dir)
    if ckpt:
        print "Restoring from", ckpt
        saver.restore(sess=session, save_path=ckpt)
    for epoch in range(NUM_EPOCHS):
        print "Starting epoch %d" % epoch
        print "Validation:"
        valid_score = do_epoch(session=session, sequences=valid_seq, mode="valid")
        if best_validation_score is None: 
            best_validation_score = valid_score
            with open("v3/test-predictions-epoch%d" % epoch, "w") as out:
                test_predictions = do_epoch(session=session, sequences=test_seq, mode="test")
                print >> out, "frame_id,steering_angle"
                for img, pred in test_predictions.items():
                    img = img.replace("challenge_2/Test-final/center/", "")
                    print >> out, "%s,%f" % (img, pred)
        if valid_score < best_validation_score:
            saver.save(session, 'v3/checkpoint-sdc-ch2')
            best_validation_score = valid_score
            print '\r', "SAVED at epoch %d" % epoch,
            with open("v3/valid-predictions-epoch%d" % epoch, "w") as out:
                result = np.float128(0.0)
                for img, stats in global_valid_predictions.items():
                    print >> out, img, stats
                    result += stats[-1]
            print "Validation unnormalized RMSE:", np.sqrt(result / len(global_valid_predictions))
            with open("v3/test-predictions-epoch%d" % epoch, "w") as out:
                test_predictions = do_epoch(session=session, sequences=test_seq, mode="test")
                print >> out, "frame_id,steering_angle"
                for img, pred in test_predictions.items():
                    img = img.replace("challenge_2/Test-final/center/", "")
                    print >> out, "%s,%f" % (img, pred)
        if epoch != NUM_EPOCHS - 1:
            print "Training"
            do_epoch(session=session, sequences=train_seq, mode="train")

Initialized
Starting epoch 0
Validation:
180 / 180 1.50270079113

Training
10423 / 10423 1.8467856622
Starting epoch 1
Validation:
180 / 180 0.874280615595
SAVED at epoch 1 Validation unnormalized RMSE: 0.0453934155637

Training
10423 / 10423 1.77418961999
Starting epoch 2
Validation:
180 / 180 0.811133286214
SAVED at epoch 2 Validation unnormalized RMSE: 0.0421147514332

Training
10423 / 10423 1.68576334939
Starting epoch 3
Validation:
180 / 180 0.800186080058
SAVED at epoch 3 Validation unnormalized RMSE: 0.0415463624164

Training
10423 / 10423 1.54967948446
Starting epoch 4
Validation:
180 / 180 0.878587929231
Training
10423 / 10423 1.32308876818
Starting epoch 5
Validation:
180 / 180 0.845009498743
Training
10423 / 10423 1.11399653788
Starting epoch 6
Validation:
180 / 180 0.723817861367
SAVED at epoch 6 Validation unnormalized RMSE: 0.0375812570687

Training
10423 / 10423 0.966845877805
Starting epoch 7
Validation:
180 / 180 0.630595047285
SAVED at epoch 7 Validation unnormalized 

KeyboardInterrupt: 

In [7]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)

checkpoint_dir = os.getcwd() + "/v3"

global_train_step = 0
global_valid_step = 0
global_valid_predictions = {}

KEEP_PROB_TRAIN = 0.25

def do_epoch(session, sequences, mode):
    global global_train_step, global_valid_step, global_valid_predictions
    test_predictions = {}
    batch_generator = BatchGenerator(sequence=sequences, seq_len=SEQ_LEN, batch_size=BATCH_SIZE)
    total_num_steps = 1 + (batch_generator.indices[1] - 1) / SEQ_LEN
    controller_final_state_gt_cur, controller_final_state_autoregressive_cur = None, None
    acc_loss = np.float128(0.0)
    for step in range(total_num_steps):
        feed_inputs, feed_targets = batch_generator.next()
        feed_dict = {inputs : feed_inputs, targets : feed_targets}
        if controller_final_state_autoregressive_cur is not None:
            feed_dict.update({controller_initial_state_autoregressive : controller_final_state_autoregressive_cur})
        if controller_final_state_gt_cur is not None:
            feed_dict.update({controller_final_state_gt : controller_final_state_gt_cur})
        if mode == "train":
            feed_dict.update({keep_prob : KEEP_PROB_TRAIN})
            summary, _, loss, controller_final_state_gt_cur, controller_final_state_autoregressive_cur = \
                session.run([summaries, optimizer, mse_autoregressive_steering, controller_final_state_gt, controller_final_state_autoregressive],
                           feed_dict = feed_dict)
            train_writer.add_summary(summary, global_train_step)
            global_train_step += 1
        elif mode == "valid":
            model_predictions, summary, loss, controller_final_state_autoregressive_cur = \
                session.run([steering_predictions, summaries, mse_autoregressive_steering, controller_final_state_autoregressive],
                           feed_dict = feed_dict)
            valid_writer.add_summary(summary, global_valid_step)
            global_valid_step += 1
            
            feed_inputs = feed_inputs[:, LEFT_CONTEXT:].flatten()
            steering_targets = feed_targets[:, :, 0].flatten()
            model_predictions = model_predictions.flatten()
            stats = np.stack([steering_targets, model_predictions, (steering_targets - model_predictions)**2])
            for i, img in enumerate(feed_inputs):
                global_valid_predictions[img] = stats[:, i]
        elif mode == "test":
            model_predictions, controller_final_state_autoregressive_cur = \
                session.run([steering_predictions, controller_final_state_autoregressive],
                           feed_dict = feed_dict)           
            feed_inputs = feed_inputs[:, LEFT_CONTEXT:].flatten()
            model_predictions = model_predictions.flatten()
            for i, img in enumerate(feed_inputs):
                test_predictions[img] = model_predictions[i]
        if mode != "test":
            acc_loss += loss
            print '\r', step + 1, "/", total_num_steps, np.sqrt(acc_loss / (step+1)),
    print
    return np.sqrt(acc_loss / total_num_steps) if mode != "test" else test_predictions
    

NUM_EPOCHS=100

best_validation_score = None
with tf.Session(graph=graph, config=tf.ConfigProto(gpu_options=gpu_options)) as session:
    session.run(tf.initialize_all_variables())
    print 'Initialized'
    ckpt = tf.train.latest_checkpoint(checkpoint_dir)
    if ckpt:
        print "Restoring from", ckpt
        saver.restore(sess=session, save_path=ckpt)
    for epoch in range(NUM_EPOCHS):
        print "Starting epoch %d" % epoch
        print "Validation:"
        valid_score = do_epoch(session=session, sequences=valid_seq, mode="valid")
        if best_validation_score is None: 
            best_validation_score = valid_score
            with open("v3/FINETUNE-test-predictions-epoch%d" % epoch, "w") as out:
                test_predictions = do_epoch(session=session, sequences=test_seq, mode="test")
                print >> out, "frame_id,steering_angle"
                for img, pred in test_predictions.items():
                    img = img.replace("challenge_2/Test-final/center/", "")
                    print >> out, "%s,%f" % (img, pred)
        if True or valid_score < best_validation_score:
            saver.save(session, 'v3/FINETUNE-checkpoint-sdc-ch2')
            best_validation_score = valid_score
            print '\r', "SAVED at epoch %d" % epoch,
            with open("v3/FINETUNE-valid-predictions-epoch%d" % epoch, "w") as out:
                result = np.float128(0.0)
                for img, stats in global_valid_predictions.items():
                    print >> out, img, stats
                    result += stats[-1]
            print "Validation unnormalized RMSE:", np.sqrt(result / len(global_valid_predictions))
            with open("v3/FINETUNE-test-predictions-epoch%d" % epoch, "w") as out:
                test_predictions = do_epoch(session=session, sequences=test_seq, mode="test")
                print >> out, "frame_id,steering_angle"
                for img, pred in test_predictions.items():
                    img = img.replace("challenge_2/Test-final/center/", "")
                    print >> out, "%s,%f" % (img, pred)
        if epoch != NUM_EPOCHS - 1:
            print "Training"
            do_epoch(session=session, sequences=train_seq_main_3, mode="train")

Initialized
Restoring from /home/ilia/nn/sdc2/udacity-driving-reader/v3/checkpoint-sdc-ch2
Starting epoch 0
Validation:
180 / 180 0.471942768313

SAVED at epoch 0 Validation unnormalized RMSE: 0.0245036814524

Training
846 / 846 1.25031223664
Starting epoch 1
Validation:
180 / 180 0.504185079126
SAVED at epoch 1 Validation unnormalized RMSE: 0.0261777304285

Training
846 / 846 1.13577142797
Starting epoch 2
Validation:
180 / 180 0.50290400244
SAVED at epoch 2 Validation unnormalized RMSE: 0.0261112159347

Training
846 / 846 1.03962368301
Starting epoch 3
Validation:
180 / 180 0.502557811881
SAVED at epoch 3 Validation unnormalized RMSE: 0.0260932414149

Training
846 / 846 1.02208957899
Starting epoch 4
Validation:
180 / 180 0.48331428505
SAVED at epoch 4 Validation unnormalized RMSE: 0.0250941006176

Training
846 / 846 1.06320347588
Starting epoch 5
Validation:
180 / 180 0.482416603701
SAVED at epoch 5 Validation unnormalized RMSE: 0.0250474920664

Training
846 / 846 0.980025817164
Sta

KeyboardInterrupt: 

In [7]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)

checkpoint_dir = os.getcwd() + "/v3"

global_train_step = 0
global_valid_step = 0
global_valid_predictions = {}

KEEP_PROB_TRAIN = 0.25

def do_epoch(session, sequences, mode):
    global global_train_step, global_valid_step, global_valid_predictions
    test_predictions = {}
    batch_generator = BatchGenerator(sequence=sequences, seq_len=SEQ_LEN, batch_size=BATCH_SIZE)
    total_num_steps = 1 + (batch_generator.indices[1] - 1) / SEQ_LEN
    controller_final_state_gt_cur, controller_final_state_autoregressive_cur = None, None
    acc_loss = np.float128(0.0)
    for step in range(total_num_steps):
        feed_inputs, feed_targets = batch_generator.next()
        feed_dict = {inputs : feed_inputs, targets : feed_targets}
        if controller_final_state_autoregressive_cur is not None:
            feed_dict.update({controller_initial_state_autoregressive : controller_final_state_autoregressive_cur})
        if controller_final_state_gt_cur is not None:
            feed_dict.update({controller_final_state_gt : controller_final_state_gt_cur})
        if mode == "train":
            feed_dict.update({keep_prob : KEEP_PROB_TRAIN})
            summary, _, loss, controller_final_state_gt_cur, controller_final_state_autoregressive_cur = \
                session.run([summaries, optimizer, mse_autoregressive_steering, controller_final_state_gt, controller_final_state_autoregressive],
                           feed_dict = feed_dict)
            train_writer.add_summary(summary, global_train_step)
            global_train_step += 1
        elif mode == "valid":
            model_predictions, summary, loss, controller_final_state_autoregressive_cur = \
                session.run([steering_predictions, summaries, mse_autoregressive_steering, controller_final_state_autoregressive],
                           feed_dict = feed_dict)
            valid_writer.add_summary(summary, global_valid_step)
            global_valid_step += 1
            
            feed_inputs = feed_inputs[:, LEFT_CONTEXT:].flatten()
            steering_targets = feed_targets[:, :, 0].flatten()
            model_predictions = model_predictions.flatten()
            stats = np.stack([steering_targets, model_predictions, (steering_targets - model_predictions)**2])
            for i, img in enumerate(feed_inputs):
                global_valid_predictions[img] = stats[:, i]
        elif mode == "test":
            model_predictions, controller_final_state_autoregressive_cur = \
                session.run([steering_predictions, controller_final_state_autoregressive],
                           feed_dict = feed_dict)           
            feed_inputs = feed_inputs[:, LEFT_CONTEXT:].flatten()
            model_predictions = model_predictions.flatten()
            for i, img in enumerate(feed_inputs):
                test_predictions[img] = model_predictions[i]
        if mode != "test":
            acc_loss += loss
            print '\r', step + 1, "/", total_num_steps, np.sqrt(acc_loss / (step+1)),
    print
    return np.sqrt(acc_loss / total_num_steps) if mode != "test" else test_predictions
    

NUM_EPOCHS=100

best_validation_score = None
with tf.Session(graph=graph, config=tf.ConfigProto(gpu_options=gpu_options)) as session:
    session.run(tf.initialize_all_variables())
    print 'Initialized'
    ckpt = tf.train.latest_checkpoint(checkpoint_dir)
    if ckpt:
        print "Restoring from", ckpt
        saver.restore(sess=session, save_path=ckpt)
    for epoch in range(NUM_EPOCHS):
        print "Starting epoch %d" % epoch
        print "Validation:"
        valid_score = do_epoch(session=session, sequences=valid_seq, mode="valid")
        if best_validation_score is None: 
            best_validation_score = valid_score
            with open("v3/FINE_TUNE_2-test-predictions-epoch%d" % epoch, "w") as out:
                test_predictions = do_epoch(session=session, sequences=test_seq, mode="test")
                print >> out, "frame_id,steering_angle"
                for img, pred in test_predictions.items():
                    img = img.replace("challenge_2/Test-final/center/", "")
                    print >> out, "%s,%f" % (img, pred)
        if True or valid_score < best_validation_score:
            saver.save(session, 'v3/FINE_TUNE_2-checkpoint-sdc-ch2-epoch%d' % epoch)
            best_validation_score = valid_score
            print '\r', "SAVED at epoch %d" % epoch,
            with open("v3/FINE_TUNE_2-valid-predictions-epoch%d" % epoch, "w") as out:
                result = np.float128(0.0)
                for img, stats in global_valid_predictions.items():
                    print >> out, img, stats
                    result += stats[-1]
            print "Validation unnormalized RMSE:", np.sqrt(result / len(global_valid_predictions))
            with open("v3/FINE_TUNE_2-test-predictions-epoch%d" % epoch, "w") as out:
                test_predictions = do_epoch(session=session, sequences=test_seq, mode="test")
                print >> out, "frame_id,steering_angle"
                for img, pred in test_predictions.items():
                    img = img.replace("challenge_2/Test-final/center/", "")
                    print >> out, "%s,%f" % (img, pred)
        if epoch != NUM_EPOCHS - 1:
            print "Training"
            do_epoch(session=session, sequences=train_seq, mode="train")

Initialized
Restoring from /home/ilia/nn/sdc2/udacity-driving-reader/v3/FINE_TUNE_2-checkpoint-sdc-ch2-epoch2
Starting epoch 0
Validation:
180 / 180 0.493319772903

SAVED at epoch 0 Validation unnormalized RMSE: 0.0256135944036

Training
10423 / 10423 0.357055723079
Starting epoch 1
Validation:
180 / 180 0.506785779906
SAVED at epoch 1 Validation unnormalized RMSE: 0.026312761425

Training
10423 / 10423 0.343080845742
Starting epoch 2
Validation:
180 / 180 0.506069515761
SAVED at epoch 2 Validation unnormalized RMSE: 0.0262755722093

Training
10423 / 10423 0.359480703785
Starting epoch 3
Validation:
180 / 180 0.490330997847
SAVED at epoch 3 Validation unnormalized RMSE: 0.0254584145878

Training
10423 / 10423 0.352017954507
Starting epoch 4
Validation:
180 / 180 0.488294982346
SAVED at epoch 4 Validation unnormalized RMSE: 0.0253527029018

Training
10423 / 10423 0.352547655156
Starting epoch 5
Validation:
180 / 180 0.515736059348
SAVED at epoch 5 Validation unnormalized RMSE: 0.0267774

KeyboardInterrupt: 